In [106]:
import pandas as pd

In [107]:
base_path = '/home/linhnm/onedrive/eabdt/data'

In [108]:
df = pd.read_csv(f'{base_path}/raw/All_Beauty.csv')

In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371344 entries, 0 to 371343
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   0143026860      371344 non-null  object 
 1   A1V6B6TNIC10QE  371344 non-null  object 
 2   1.0             371344 non-null  float64
 3   1424304000      371344 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 11.3+ MB


In [110]:
all_beauty = pd.read_json(f'{base_path}/raw/All_Beauty_5.json', lines=True)

In [111]:
all_beauty.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5,True,"09 1, 2016",A3CIUOJXQ5VDQ2,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Shelly F,As advertised. Reasonably priced,Five Stars,1472688000,NaN,NaN
1,5,True,"11 14, 2013",A3H7T87S984REU,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",houserules18,Like the oder and the feel when I put it on my...,Good for the face,1384387200,NaN,NaN
2,1,True,"08 18, 2013",A3J034YH7UG4KT,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Adam,I bought this to smell nice after I shave. Wh...,Smells awful,1376784000,NaN,NaN
3,5,False,"05 3, 2011",A2UEO5XR3598GI,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Rich K,HEY!! I am an Aqua Velva Man and absolutely lo...,Truth is There IS Nothing Like an AQUA VELVA MAN.,1304380800,25,NaN
4,5,True,"05 6, 2011",A3SFRT223XXWF7,B00006L9LC,{'Size:': ' 200ml/6.7oz'},C. C. Christian,If you ever want to feel pampered by a shampoo...,Bvlgari Shampoo,1304640000,3,NaN


In [112]:
from sklearn.feature_extraction.text import CountVectorizer

In [113]:
text = ["As advertised. Reasonably priced	"]
# create the transform
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# summarize
print(vectorizer.vocabulary_)
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape)
print(type(vector))
print(vector.toarray())

{'as': 1, 'advertised': 0, 'reasonably': 3, 'priced': 2}
(1, 4)
<class 'scipy.sparse._csr.csr_matrix'>
[[1 1 1 1]]


In [114]:
Xy = all_beauty[['reviewText', 'overall']]
Xy = Xy.dropna()

In [120]:
Xy['overall'].shape

(5264,)

In [121]:
X = Xy['reviewText']
y = Xy['overall']

In [122]:
X.info()

<class 'pandas.core.series.Series'>
Index: 5264 entries, 0 to 5268
Series name: reviewText
Non-Null Count  Dtype 
--------------  ----- 
5264 non-null   object
dtypes: object(1)
memory usage: 82.2+ KB


In [98]:
y.info()

<class 'pandas.core.series.Series'>
Index: 5264 entries, 0 to 5268
Series name: overall
Non-Null Count  Dtype
--------------  -----
5264 non-null   int64
dtypes: int64(1)
memory usage: 82.2 KB


In [69]:
vectorizer = CountVectorizer()
X_counts = vectorizer.fit_transform(X)

In [70]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [71]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
  ])

In [72]:
text_clf.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [73]:
text_clf.score(X_test, y_test)

0.8879392212725546

In [123]:
from sentence_transformers import SentenceTransformer
mpnet_model = SentenceTransformer('all-mpnet-base-v2')
miniLM_model = SentenceTransformer('all-MiniLM-L6-v2')



In [127]:
# X_vectorize_mpnet = Xy['reviewText'].apply(lambda row : mpnet_model.encode(row))
X_vectorize_miniLM = Xy['reviewText'].apply(lambda row : miniLM_model.encode(row))

In [153]:
X_vectorize_miniLM = pd.DataFrame(X_vectorize_miniLM.values.tolist(), index=X_vectorize_miniLM.index)

In [154]:
X_vectorize_miniLM.shape

(5264, 384)

In [155]:
import numpy as np
X_vectorize_miniLM_np = X_vectorize_miniLM.to_numpy()
np.save(f'{base_path}/vectorize/All_Beauty/X_miniLM.npy', X_vectorize_miniLM_np)

In [156]:
y.to_csv(f'{base_path}/vectorize/All_Beauty/label.csv', index=False)

In [157]:
X_vectorize_miniLM_train, X_vectorize_miniLM_test, y_train, y_test = train_test_split(X_vectorize_miniLM, y, test_size=0.2, random_state=42)

In [158]:
X_vectorize_miniLM_train.shape

(4211, 384)

In [160]:
# clf = MultinomialNB().fit(X_vectorize_miniLM_train, y_train)

ValueError: Negative values in data passed to MultinomialNB (input X)

In [161]:
from sklearn import svm

In [162]:
clf_svm = svm.SVC()
clf_svm.fit(X_vectorize_miniLM_train, y_train)

SVC()

In [163]:
clf_svm.score(X_vectorize_miniLM_test, y_test)

0.9506172839506173